## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ranong,TH,9.9658,98.6348,75.13,86,100,9.01,overcast clouds
1,1,Mar Del Plata,AR,-38.0023,-57.5575,48.09,60,0,0.00,clear sky
2,2,Celestun,MX,20.8667,-90.4000,81.81,80,45,13.58,light rain
3,3,Fortuna,US,40.5982,-124.1573,63.09,82,100,3.00,overcast clouds
4,4,Hilo,US,19.7297,-155.0900,86.29,81,100,6.91,overcast clouds
5,5,Albany,US,42.6001,-73.9662,81.68,51,40,4.83,scattered clouds
6,6,Dikson,RU,73.5069,80.5464,33.04,99,100,4.32,overcast clouds
7,7,Port-Cartier,CA,50.0334,-66.8654,67.68,66,33,8.14,scattered clouds
8,8,Dargaville,NZ,-35.9333,173.8833,60.66,78,45,11.65,light rain
9,9,Codrington,AU,-38.2667,141.9667,50.83,71,100,10.56,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ranong,TH,9.9658,98.6348,75.13,86,100,9.01,overcast clouds
2,2,Celestun,MX,20.8667,-90.4000,81.81,80,45,13.58,light rain
4,4,Hilo,US,19.7297,-155.0900,86.29,81,100,6.91,overcast clouds
5,5,Albany,US,42.6001,-73.9662,81.68,51,40,4.83,scattered clouds
10,10,Agua Verde,MX,22.9000,-106.0500,84.29,68,24,13.13,few clouds
18,18,Pakxan,LA,18.3778,103.6601,76.96,93,100,1.43,overcast clouds
20,20,Vaini,TO,-21.2000,-175.2000,77.16,88,75,6.91,broken clouds
24,24,Iranshahr,IR,27.2025,60.6848,88.88,41,16,0.96,few clouds
32,32,Faanui,PF,-16.4833,-151.7500,79.14,73,22,21.72,light rain
33,33,Sinnamary,GF,5.3833,-52.9500,76.77,87,50,6.42,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ranong,TH,75.13,overcast clouds,9.9658,98.6348,
2,Celestun,MX,81.81,light rain,20.8667,-90.4000,
4,Hilo,US,86.29,overcast clouds,19.7297,-155.0900,
5,Albany,US,81.68,scattered clouds,42.6001,-73.9662,
10,Agua Verde,MX,84.29,few clouds,22.9000,-106.0500,
18,Pakxan,LA,76.96,overcast clouds,18.3778,103.6601,
20,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
24,Iranshahr,IR,88.88,few clouds,27.2025,60.6848,
32,Faanui,PF,79.14,light rain,-16.4833,-151.7500,
33,Sinnamary,GF,76.77,scattered clouds,5.3833,-52.9500,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
        
        


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ranong,TH,75.13,overcast clouds,9.9658,98.6348,Numsai Khaosuay Resort Ranong
2,Celestun,MX,81.81,light rain,20.8667,-90.4000,Posada Lilia
4,Hilo,US,86.29,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
5,Albany,US,81.68,scattered clouds,42.6001,-73.9662,NaN
10,Agua Verde,MX,84.29,few clouds,22.9000,-106.0500,NaN
18,Pakxan,LA,76.96,overcast clouds,18.3778,103.6601,Kong Kham Pheng Guesthouse
20,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
24,Iranshahr,IR,88.88,few clouds,27.2025,60.6848,نیک سرشت
32,Faanui,PF,79.14,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
33,Sinnamary,GF,76.77,scattered clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ranong,TH,75.13,overcast clouds,9.9658,98.6348,Numsai Khaosuay Resort Ranong
2,Celestun,MX,81.81,light rain,20.8667,-90.4000,Posada Lilia
4,Hilo,US,86.29,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
5,Albany,US,81.68,scattered clouds,42.6001,-73.9662,NaN
10,Agua Verde,MX,84.29,few clouds,22.9000,-106.0500,NaN
18,Pakxan,LA,76.96,overcast clouds,18.3778,103.6601,Kong Kham Pheng Guesthouse
20,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
24,Iranshahr,IR,88.88,few clouds,27.2025,60.6848,نیک سرشت
32,Faanui,PF,79.14,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
33,Sinnamary,GF,76.77,scattered clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ranong,TH,75.13,overcast clouds,9.9658,98.6348,Numsai Khaosuay Resort Ranong
2,Celestun,MX,81.81,light rain,20.8667,-90.4000,Posada Lilia
4,Hilo,US,86.29,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
18,Pakxan,LA,76.96,overcast clouds,18.3778,103.6601,Kong Kham Pheng Guesthouse
20,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
24,Iranshahr,IR,88.88,few clouds,27.2025,60.6848,نیک سرشت
32,Faanui,PF,79.14,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
33,Sinnamary,GF,76.77,scattered clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
34,Kahului,US,86.02,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
36,Dicabisagan,PH,85.10,overcast clouds,17.0818,122.4157,Nancy Montanes Bernardo


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))